In [1]:
%load_ext autoreload
%autoreload 2
import pprint, json, math, os, sys
import fitz, pdfplumber, ocrmypdf
import pandas as pd
import numpy as np
from collections import defaultdict

dir_path = "C:\\Users\\Kaustubh.keny\\Projects\\office-work\\mywork-repo"
fund_path = "C:\\Users\\Kaustubh.keny\\Projects\\Mar 25"

# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path = "E:\\PDFDrive\\Mar25"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.parse_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def extract_clipped_text_all_pages(pdf_path, clip_coords):
    results = {}
    doc = fitz.open(pdf_path)
    clip_rect = fitz.Rect(*clip_coords)
    try:
        for page_number, page in enumerate(doc):
            text = page.get_text("text", clip=clip_rect).strip()
            results[page_number] = text
    finally:
        doc.close()
    return results

In [2]:
sample_path = mutual_fund["Unifi Mutual Fund"]

In [6]:
lines = [
    ((270, 0), (270, 812)),# Vertical line
    ((360, 0), (360, 812)),
    ((0, 560), (812, 560))
]
pages = [12, 14,16]
bboxes = [ [0, 55, 270, 600],
        [270, 55, 580, 600],] #[(0, 85, 180, 812),(180, 85, 360, 812),(0,100,270,812),(0,100,350,812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dir_path +dry_path)

Modified PDF saved to: C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\data\output\DryRun.pdf


In [ ]:

clip_coords = (484, 50,600, 100)  # (x0, y0, x1, y1)

clipped_texts = extract_clipped_text_all_pages(sample_path, clip_coords)

for page_num, text in clipped_texts.items():
    print(f"Page {page_num}:\n{text}\n{'-'*40}")


In [ ]:
def get_proper_fund_names(path: str):
    pattern = "(HDFC.*?(?:FUND|Fund|ETF|FO?o?F)\\s*(?:of Funds?|.+?Plan|Fund of Funds?|Fund)?)"
    title = {}   
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 400, 60)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\-\\(\\).,]+|\u2028", "", text).strip()
            print(text)
            if matches := re.findall(pattern, text, re.DOTALL):
                title[pgn] = " ".join([_ for _ in matches[0].strip().split(" ") if _])
                print(pgn,matches[0])
    return title
title = get_proper_fund_names(sample_path)

In [43]:
import os ,json 
import pandas as pd

full_name = set()
split_name = set()
MANAGER_REGEX = FundRegex().MANAGER_STOP_WORDS
for a,b,files in os.walk(r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\sql_learn\json\MAR25DATA"):
    for paths in files:
        sample_path = os.path.join(os.getcwd(),"..","sql_learn","json","MAR25DATA",paths)
        # print(sample_path)
        try:
            with open(sample_path, "r", encoding="utf-8") as file:
                data = json.load(file)
                for k,scheme in data.items():
                    # print(k)
                    if "fund_manager" in scheme:
                        for entry in scheme['fund_manager']:
                            # print(entry['name'])
                            name = entry['name']
                            for regex_ in MANAGER_REGEX:
                                name = re.sub(f"\\b{regex_}\\b|[^A-Za-z\\s0-9]+","",name, re.IGNORECASE)
                                name = re.sub(r"\s+"," ",name)
                            full_name.add(name)
                            printthis = name if name.strip() else "EMPTY"
                            # print(f"<<{printthis}>>")
                            if printthis == "EMPTY":
                                print(k,printthis,entry['name'])
                            # split_name.add(name.split(' '))
                    # print(scheme.keys())
        except Exception as e:
            print(f"NEVER MIND {e}")
    # print(files)
        

C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_20632\1373465500.py:21: DeprecationWarning: 'count' is passed as positional argument
  name = re.sub(f"\\b{regex_}\\b|[^A-Za-z\\s0-9]+","",name, re.IGNORECASE)


CANARA ROBECO BLUE CHIP EQUITY FUND (CRBCEF) EMPTY To Benefit
CANARA ROBECO BLUE CHIP EQUITY FUND (CRBCEF) EMPTY To Benefit
CANARA ROBECO EMERGING EQUITIES (CREE) EMPTY India Ltd
CANARA ROBECO EMERGING EQUITIES (CREE) EMPTY India Ltd
CANARA ROBECO SMALL CAP FUND (CRSCF) EMPTY Of Full
CANARA ROBECO SMALL CAP FUND (CRSCF) EMPTY To Invest
CANARA ROBECO SMALL CAP FUND (CRSCF) EMPTY Of Full
CANARA ROBECO SMALL CAP FUND (CRSCF) EMPTY To Invest
CANARA ROBECO INFRASTRUCTURE (CRI) EMPTY To Have
CANARA ROBECO VALUE FUND (CRVF) EMPTY To Generate
CANARA ROBECO VALUE FUND (CRVF) EMPTY To Benefit
CANARA ROBECO VALUE FUND (CRVF) EMPTY To Generate
CANARA ROBECO VALUE FUND (CRVF) EMPTY To Benefit
CANARA ROBECO LIQUID FUND (CRL) EMPTY To Provide
CANARA ROBECO LIQUID FUND (CRL) EMPTY To Provide
CANARA ROBECO DYNAMIC BOND FUND (CRDBF) EMPTY At Generating
CANARA ROBECO BANKING AND PSU DEBT FUND (CRBPDF) EMPTY To Capture
CANARA ROBECO BANKING AND PSU DEBT FUND (CRBPDF) EMPTY To Capture
CANARA ROBECO BALANCE

In [ ]:
FundRegex().MANAGER_STOP_WORDS[:10]

In [16]:
data = Helper.get_all_pdf_data(sample_path)

In [ ]:
import camelot
def get_something(path: str):
    pattern = r"^(REDEMPTION PROCEEDS|FEATURES|ASSET ALLOCATION|FUND MANAGER|SCHEME|Sr\. No\.)$"

    with fitz.open(path) as doc:
        exists = defaultdict(int)
        for pgn, page in enumerate(doc):
            page_text = [t.strip() for t in page.get_text().split("\n")]
            for text in page_text:
                if re.match(pattern,text):
                    exists[pgn]+=1
                    
    
    return [str(pgn+1) for pgn, count in exists.items() if count > 4] #camelot starts pages from 1

pages = get_something(sample_path)

imp_pages = ",".join(pages)                

In [ ]:
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="stream", table_areas= ["30,50,612,812"],column_tol = 4, split_text = True) #["30,50,612,812"]   

In [ ]:
import camelot
import pandas as pd
import numpy as np

def clean_column_name(x):
    cleaned = re.sub(r"[^A-Za-z0-9\s]", "", str(x), flags=re.IGNORECASE) #Corrected re.sub.
    cleaned = "_".join(cleaned.split())
    return cleaned

with pd.ExcelWriter("cleaned_tables.xlsx", engine="xlsxwriter") as writer:
    for i, table in enumerate(tables):
        df = table.df
        df = df.map(lambda x: " ".join(str(x).split("\n")).strip())
        df.replace("", np.nan, inplace=True)
        df.iloc[2:, 0] = df.iloc[2:, 0].ffill()
        df = df.iloc[1:, :]
        df.columns = df.iloc[0, :].apply(clean_column_name)
        # print(df.columns)
        df = df.iloc[1:, :]
        sheet_name = f"Table_{i+1}"

        # workbook = writer.book
        # worksheet = writer.sheets[sheet_name]

        # row = 1
        # last_value = None
        # start_row = None

        # for j, value in enumerate(df.iloc[:, 0], start=1):
        #     if pd.notna(value):  # New group found
        #         if last_value is not None and start_row is not None:
        #             worksheet.merge_range(start_row, 0, row - 1, 0, last_value)  # Merge previous block
        #         last_value = value
        #         start_row = row
        #     row += 1

        # # Merge last group
        # if last_value is not None and start_row is not None:
        #     worksheet.merge_range(start_row, 0, row - 1, 0, last_value)
    
        

print("Cleaned tables saved with merged spanning rows!")


In [ ]:
hello =  {
    "number": 0,
    "type": 0,
    "bbox": (0,0,0,0), #406.72119140625, 439.4930419921875, 565.697265625, 484.5830383300781
    "lines": [
        {
            "spans": [
                {
                    "size": 30.0,
                    "flags": 20,
                    "font": "Montserrat-Regular", #set this
                    "color": -1, #set this
                    "ascender": 1.0429999828338623,
                    "descender": -0.2619999945163727,
                    "text": "DUMMYDUMMYDUMMYDUMMY",
                    "origin": (406.72119140625, 458.26702880859375),
                    "bbox": (0,0,0,0), #406.72119140625,439.4930419921875,565.697265625,462.9830322265625,
                }
            ],
            "wmode": 0,
            "dir": (1.0, 0.0),
            "bbox": (0,0,0,0), #406.72119140625,439.4930419921875,565.697265625,462.9830322265625,
        },
        
    ],
},  